In [ ]:
# importing dependencies 

import numpy as np 
import HGF
import pandas as pd
from scipy.stats import lognorm
import os,sys
import matplotlib.pyplot as plt

# importing HGF library 
import HGF.hgf_config
import HGF.hgf_fit

# Extracting the parent dir in order to append it to the python path 
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))  # Adjust if needed

# Add it to the Python path
sys.path.append(parent_dir)

# importing dependencies 
from PAM.LNR.lnr_hgf_config import lnr_hgf_config



In [2]:
# load the trial list -- Specify the path -- 
u = pd.read_csv(r'C:\Users\fraca\Desktop\PAM\Pam-master\u.csv')
u = u.values
u = u.flatten()


## Section 1: Simulate dataset

In [ ]:

# Set parameters


om2 = -4 # learning rate
a = -0.53 # mu of the lognormal distribution
b_val = -0.47 # validity effect on a
b = -0.19 # belief effect
sigma = .25 # LNR variance
Ter = 0 # non-decision time

# Simulate Beliefs
bo = HGF.hgf_fit.fitModel([], u,HGF.hgf_config.ehgf_binary_config, HGF.hgf_config.bayes_optimal_binary_config)
priormus = bo['p_prc']['p'] 
priormus[-2]=om2
esim = HGF.hgf_sim.simModel(u,HGF.hgf.ehgf_binary,priormus)

# Extract trial-wise beliefs about u
muhat = esim['traj']['mu_hat'][:,0]


Ignored trials: []
Irregular trials: []

Initializing optimization run...

Optimization terminated successfully.
         Current function value: 265.698074
         Iterations: 6
         Function evaluations: 21
         Gradient evaluations: 7


RESULTS:

Parameter estimates - perceptual model:
 mu_0: 	 [nan  0.  1.]
 sa_0: 	 [nan 0.1 1. ]
 rho: 	 [nan  0.  0.]
 ka: 	 [1. 1.]
 om: 	 [        nan -3.57818887]
 th: 	 0.6495383397499885

MODEL QUALITY:
 LME: 	 -265.41135491471664 		 (more is better)
 AIC: 	 528.1669974755891 		 (less is better)
 BIC: 	 534.7636322086852 		 (less is better)
Ignored trials: []


C:\Users\fraca\HGF\HGF\hgf.py:38: RuntimeWarning: invalid value encountered in multiply
  pi = np.empty((n, l)) * np.nan         # pi representation
C:\Users\fraca\HGF\HGF\hgf.py:40: RuntimeWarning: invalid value encountered in multiply
  pi_hat = np.empty((n, l)) * np.nan     # pi^ quantity
C:\Users\fraca\HGF\HGF\hgf.py:37: RuntimeWarning: invalid value encountered in multiply
  mu = np.empty((n, l)) * np.nan         # mu represnetation
C:\Users\fraca\HGF\HGF\hgf.py:39: RuntimeWarning: invalid value encountered in multiply
  mu_hat = np.empty((n, l)) * np.nan     # mu^ quantity
C:\Users\fraca\HGF\HGF\hgf.py:41: RuntimeWarning: invalid value encountered in multiply
  v = np.empty((n, l)) * np.nan


## Section 2: Simulate Responses

In [ ]:
# SIMULATE RESPONSES 

mu_c1 = a + np.multiply(b_val,(u==1).astype(float)) + np.multiply(b,(.5-muhat))
mu_c0 = a + np.multiply(b_val,(u==0).astype(float)) + np.multiply(b,(.5-(1-muhat)))


# Initialize arrays for response time and response
rt = np.full((len(u), 1), np.nan)
resp = np.full((len(u), 1), np.nan)


# Loop over the trial list
for n in range(len(u)):
    # Define probability distributions for the two accumulators
    x = np.arange(0.01, 3.01, 0.01)  # Similar to 0.01:0.01:3 in MATLAB

    # Log-normal PDF for the first accumulator
    P1_pdf = lognorm.pdf(x, s=sigma, scale=np.exp(mu_c1[n]))
    # Sample from the first accumulator probability distribution
    P1 = np.random.choice(x, p=P1_pdf/P1_pdf.sum())

    # Log-normal PDF for the second accumulator
    P2_pdf = lognorm.pdf(x, s=sigma, scale=np.exp(mu_c0[n]))
    # Sample from the second accumulator probability distribution
    P2 = np.random.choice(x, p=P2_pdf/P2_pdf.sum())
    # Collect the sampled probabilities
    P = [P1, P2]
    # Take the minimum probability (response time) and its index (response)
    rt[n, 0], resp[n, 0] = min(P), np.argmin(P) + 1

# Add Ter to all response times
rt += Ter

# Set the response of the second accumulator to 0
resp[resp == 2] = 0

## Section 3: Fit LNR HGF


In [ ]:
# Configure Model
overwrite_option_mus = {'prc_model' : {'ommu' : bo['p_prc']['om'] }}
obs_model =  lnr_hgf_config

# Combine rt and resp for the current subject
y = np.column_stack((rt, resp))  

# Fit the model 
m = HGF.hgf_fit.fitModel(y, u,HGF.hgf_config.ehgf_binary_config , obs_model, opt_model=HGF.hgf_config.quasinewton_optim_config,overwrite_opt=overwrite_option_mus)






Ignored trials: []
Irregular trials: []

Initializing optimization run...



C:\Users\fraca\HGF\HGF\hgf.py:43: RuntimeWarning: invalid value encountered in multiply
  da = np.empty((n, l)) * np.nan         # prediction errors


Optimization terminated successfully.
         Current function value: -290.697013
         Iterations: 18
         Function evaluations: 189
         Gradient evaluations: 27


RESULTS:

Parameter estimates - perceptual model:
 mu_0: 	 [nan  0.  1.]
 sa_0: 	 [nan 0.1 1. ]
 rho: 	 [nan  0.  0.]
 ka: 	 [1. 1.]
 om: 	 [        nan -2.40122752]
 th: 	 3.050220280768524

Parameter estimates - observation model:
 a: 	 -0.521495655426998
 b_val: 	 -0.4798978077806823
 b: 	 -0.1951405406466159
 sigma: 	 0.2479593605504163
 Ter: 	 nan

MODEL QUALITY:
 LME: 	 278.7512827759781 		 (more is better)
 AIC: 	 -589.1923447279057 		 (less is better)
 BIC: 	 -589.4802534079053 		 (less is better)


c:\Users\fraca\HGF\.venv\Lib\site-packages\statsmodels\tsa\stattools.py:702: RuntimeWarning: invalid value encountered in divide
  acf = avf[: nlags + 1] / avf[0]
